## Start

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import glob
from scipy import optimize
import importlib

In [2]:
project_path = Path.cwd().parent

## load_data

### get_latest_position_file

In [3]:
data_dir = f'{project_path}/data'

In [4]:
files = glob.glob(os.path.join(data_dir, 'Portfolio_Positions_*.csv'))
files

['/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-06-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Aug-05-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Nov-16-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-29-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-02-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_May-06-2025.csv']

In [5]:
latest_file = None
latest_date = None

for f in files:
    basename = os.path.basename(f)
    date_part = basename.replace('Portfolio_Positions_', '').replace('.csv', '')
    try:
        date_obj = datetime.strptime(date_part, '%b-%d-%Y')
        if latest_date is None or date_obj > latest_date:
            latest_date = date_obj
            latest_file = f
    except ValueError:
        continue
latest_date = pd.to_datetime(latest_date)
[latest_file, latest_date]

['/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv',
 Timestamp('2025-12-31 00:00:00')]

### clean_positions

In [6]:
from support_functions.data_loader import get_latest_position_file, clean_currency

data_dir = f'{project_path}/data'
pos_file, pos_date = get_latest_position_file(data_dir)
positions_df = pd.read_csv(pos_file, index_col=False)
positions_df = positions_df.dropna(subset=['Account Name'])
cols_to_clean = [
    'Last Price', 'Current Value', 
    'Cost Basis Total', 'Today\'s Gain/Loss Dollar', 
    'Total Gain/Loss Dollar'
]

In [7]:
for col in cols_to_clean:
    if col in positions_df.columns:
        positions_df[col] = positions_df[col].apply(clean_currency)

# Clean Quantity (remove match for formatting issues if any)
if 'Quantity' in positions_df.columns:
    positions_df['Quantity'] = pd.to_numeric(positions_df['Quantity'], errors='coerce').fillna(0)

In [8]:
positions_df['Account Name'].unique()

array(['Individual', 'ERNST & YOUNG 401(K)',
       'Cash Management (Individual)', 'Health Savings Account'],
      dtype=object)

### load_transactions

In [9]:
data_dir = f'{project_path}/data'
max_cols=14
hist_files = glob.glob(os.path.join(data_dir, 'Accounts_History_*.csv'))
transactions_dfs = []
print(f"Found {len(hist_files)} history files.")

for f in hist_files:
    df = pd.read_csv(f, header=0, usecols=range(max_cols))
    transactions_dfs.append(df)

transactions_df = pd.concat(transactions_dfs, ignore_index=True)

Found 4 history files.


In [10]:
transactions_df.columns

Index(['Run Date', 'Account', 'Account Number', 'Action', 'Symbol',
       'Description', 'Type', 'Price ($)', 'Quantity', 'Commission ($)',
       'Fees ($)', 'Accrued Interest ($)', 'Amount ($)', 'Settlement Date'],
      dtype='object')

In [12]:
transactions_df['Account'].unique()

array(['Individual', 'Health Savings Account',
       'Cash Management (Individual)', 'ERNST & YOUNG 401(K)', nan],
      dtype=object)

In [13]:
[transactions_df.iloc[292]['Run Date'],transactions_df.iloc[228]['Run Date']]

['01/31/2024', '04/25/2024']

### clean_transactions

In [14]:
from support_functions import data_loader
importlib.reload(data_loader)
from support_functions.data_loader import load_transactions, clean_currency
data_dir = f'{project_path}/data'
transactions_df = load_transactions(data_dir)

Found 4 history files.


In [15]:
transactions_df.columns

Index(['Run Date', 'Account', 'Account Number', 'Action', 'Symbol',
       'Description', 'Type', 'Price ($)', 'Quantity', 'Commission ($)',
       'Fees ($)', 'Accrued Interest ($)', 'Amount ($)', 'Settlement Date'],
      dtype='object')

In [16]:
transactions_df['Account'].unique()

array(['Individual', 'Health Savings Account',
       'Cash Management (Individual)', 'ERNST & YOUNG 401(K)', nan],
      dtype=object)

In [18]:
for col in transactions_df.columns:
    if (
        (transactions_df[col].dtype == 'object' ) or 
        (transactions_df[col].dtype == 'string')
    ):
        transactions_df[col] = transactions_df[col].str.strip()   
# Standardize dates
transactions_df['Run Date'] = pd.to_datetime(transactions_df['Run Date'], errors='coerce')
# Sometimes 'Settlement Date' exists
if 'Settlement Date' in transactions_df.columns:
        transactions_df['Settlement Date'] = pd.to_datetime(transactions_df['Settlement Date'], errors='coerce')
    


# Clean numeric columns
hist_numeric_cols = [
    'Amount ($)', 'Price ($)', 'Quantity', 
    'Commission ($)', 'Fees ($)', 'Accrued Interest ($)'
]
for col in hist_numeric_cols:
    if col in transactions_df.columns:
        transactions_df[col] = transactions_df[col].apply(clean_currency)
    
# Sort by date
transactions_df = transactions_df.sort_values('Run Date')

In [26]:
transactions_df['Amount ($)'].unique()

array(['Amount ($)', '-198.36', '-3.26', '0.2', '198.36', '3.26',
       '159.61', '40', '132.4', '308.95', '30000', '-342.04', '342.04',
       '-1642.64', '1642.64', '-11.12', '-1065.29', '11.12', '1065.29',
       '4.85', '35.4', '4', '-98925.69', '-2100', '24.9', '20000',
       '-20000', '-40000', '150000', '-99486.67', '18.3', '-120.16',
       '120.16', '-2.05', '50000', '2.05', '17.5', '-19861.24', '-3200',
       '-29840.57', '5', '7', '105.92', '247.16', '12.5', '-211.35',
       '211.35', '-1.7', '0.22', '1.7', '-19891.97', '-49432.51', '70000',
       '-4600', '-49429.35', '-19926.11', '-49424.93', '-6.64', '-241.99',
       '6.64', '241.99', '-1000', '5.33', '12.44', '11.1', '0.08',
       '-1.27', '0.28', '-2.98', '-83.8', '83.8', '-3.44', '0.23',
       '-29658.75', '3.44', '-29838.3', '10000', '33.4', '25000', '9.62',
       '22.5', '-9941.9', '17.1', '-260.33', '-2.08', '0.24', '260.33',
       '2.08', '-49699.58', '-49363.67', '129.32', '301.76', '100000',
       '-49

In [5]:
transactions_df['Account'].unique()

array(['Individual', 'Cash Management (Individual)',
       'Health Savings Account', nan], dtype=object)

In [41]:
transactions_df[transactions_df['Account']=='Health Savings Account'].head()

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
142,2024-01-17,Health Savings Account,241802439,PARTIC CONTR CURRENT PARTICIPANT CUR YR (Cash),,No Description,Cash,0.0,0.0,0.0,0.0,0.0,159.61,NaT
156,2024-01-31,Health Savings Account,241802439,INTEREST EARNED FDIC INSURED DEPOSIT AT LEADER...,QKZCQ,FDIC INSURED DEPOSIT AT LEADER BANK HSA,Cash,0.0,0.0,0.0,0.0,0.0,0.16,NaT
141,2024-02-01,Health Savings Account,241802439,PARTIC CONTR CURRENT PARTICIPANT CUR YR (Cash),,No Description,Cash,0.0,0.0,0.0,0.0,0.0,159.61,NaT
140,2024-02-14,Health Savings Account,241802439,PARTIC CONTR CURRENT PARTICIPANT CUR YR (Cash),,No Description,Cash,0.0,0.0,0.0,0.0,0.0,159.61,NaT
157,2024-02-26,Health Savings Account,241802439,EXCHANGED TO FDRXX FIDELITY GOVERNMENT CASH RE...,FDRXX,FIDELITY GOVERNMENT CASH RESERVES,Cash,0.0,0.0,0.0,0.0,0.0,0.00,NaT


### load_data

In [ ]:
from support_functions import analysis
importlib.reload(analysis)
from support_functions.data_loader import (
    get_latest_position_file, clean_positions,
    load_transactions, clean_transactions
)

In [4]:
data_dir = f'{project_path}/data'
pos_file, pos_date = get_latest_position_file(data_dir)
print(f"Loading positions from: {pos_file} (Date: {pos_date.strftime('%Y-%m-%d')})")
positions_df = pd.read_csv(pos_file, index_col=False)

positions_df = clean_positions(positions_df)


Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)


In [5]:
transactions_df = load_transactions(data_dir)
    
transactions_df = clean_transactions(transactions_df)

Found 4 history files.


In [88]:
# transactions_df[transactions_df['Symbol'] == 'AAPL']

### Clean Data

In [89]:
from support_functions.data_loader import load_data

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [90]:
ref_df = positions_df[['Symbol', 'Description']].dropna().drop_duplicates()
desc_to_sym = dict(zip(ref_df['Description'].str.strip(), ref_df['Symbol'].str.strip()))

In [93]:
mask_missing_sym = transactions_df['Symbol'].isna() | (transactions_df['Symbol'] == '')
mapped_syms = transactions_df.loc[mask_missing_sym, 'Description'].str.strip().map(desc_to_sym)
transactions_df.loc[mask_missing_sym, 'Symbol'] = mapped_syms.fillna(transactions_df.loc[mask_missing_sym, 'Symbol'])

In [100]:
desc_to_sym['SP 500 INDEX PL CL F']

'84679P173'

In [ ]:
transactions_df.loc[295]['Description']
SP 500 INDEX PL CL E
SP 500 INDEX PL CL F

'SP 500 INDEX PL CL E'

In [95]:
transactions_df[transactions_df['Description']=='SP 500 INDEX PL CL E']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Price ($),Quantity,Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Asset Type
295,2024-01-26,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.070,0.0,0.0,0.0,0.0,242.30,NaT,Stock
284,2024-02-09,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.041,0.0,0.0,0.0,0.0,242.30,NaT,Stock
277,2024-02-23,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.027,0.0,0.0,0.0,0.0,242.30,NaT,Stock
264,2024-03-08,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.020,0.0,0.0,0.0,0.0,242.30,NaT,Stock
260,2024-03-22,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,0.997,0.0,0.0,0.0,0.0,242.30,NaT,Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,2025-11-14,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,9.770,0.0,0.0,0.0,0.0,3118.98,NaT,Stock
360,2025-11-28,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,0.982,0.0,0.0,0.0,0.0,318.98,NaT,Stock
346,2025-12-12,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,0.985,0.0,0.0,0.0,0.0,318.98,NaT,Stock
336,2025-12-17,ERNST & YOUNG 401(K),86964,Change In Market Value,NaN,SP 500 INDEX PL CL E,NaN,0.000,0.0,0.0,0.0,0.0,2395.56,NaT,Stock


## analyze_entity_performance

### filter_transactions

#### ERNST & YOUNG 401(K) 86964

In [82]:
from support_functions.data_loader import load_data

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = None
account_num = '86964'

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [83]:
df = transactions_df.copy()
funding_patterns = [
    'ELECTRONIC FUNDS TRANSFER', 'CHECK RECEIVED', 'DEPOSIT', 'WIRE', 
    'BILL PAY', 'CONTRIB', 'PARTIC CONTR'
]
mask_account = (df['Account Number'] == account_num)
mask_pattern = df['Action'].str.upper().apply(lambda x: any(p in str(x) for p in funding_patterns))

df = df[mask_account & mask_pattern]

In [84]:
transactions_df['Account Number'].unique()

array(['Z23390746', 'Z06872898', '241802439', '86964', nan], dtype=object)

In [86]:
account_num = '86964'
symbol = None
df = transactions_df.copy()
funding_patterns = [
    'ELECTRONIC FUNDS TRANSFER', 'CHECK RECEIVED', 'DEPOSIT', 'WIRE', 
    'BILL PAY', 'CONTRIB', 'PARTIC CONTR'
]
mask_account = (df['Account Number'] == account_num)
df = df[mask_account]

In [87]:
df

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Price ($),Quantity,Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Asset Type
294,2024-01-26,ERNST & YOUNG 401(K),86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.716,0.0,0.0,0.0,0.0,103.85,NaT,Stock
295,2024-01-26,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.070,0.0,0.0,0.0,0.0,242.30,NaT,Stock
284,2024-02-09,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.041,0.0,0.0,0.0,0.0,242.30,NaT,Stock
283,2024-02-09,ERNST & YOUNG 401(K),86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.542,0.0,0.0,0.0,0.0,103.85,NaT,Stock
277,2024-02-23,ERNST & YOUNG 401(K),86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.027,0.0,0.0,0.0,0.0,242.30,NaT,Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,2025-12-17,ERNST & YOUNG 401(K),86964,Change In Market Value,NaN,SP 500 INDEX PL CL E,NaN,0.000,0.0,0.0,0.0,0.0,2395.56,NaT,Stock
337,2025-12-17,ERNST & YOUNG 401(K),86964,Exchange Out,NaN,SP 500 INDEX PL CL E,NaN,-60.961,0.0,0.0,0.0,0.0,-19450.82,NaT,Stock
334,2025-12-19,ERNST & YOUNG 401(K),86964,Dividend,NaN,FID BLUE CHIP GR K6,NaN,0.215,0.0,0.0,0.0,0.0,9.50,NaT,Stock
322,2025-12-26,ERNST & YOUNG 401(K),86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.047,0.0,0.0,0.0,0.0,136.71,NaT,Stock


In [85]:
account_num = '86964'
symbol = None
df = transactions_df.copy()
funding_patterns = [
    'ELECTRONIC FUNDS TRANSFER', 'CHECK RECEIVED', 'DEPOSIT', 'WIRE', 
    'BILL PAY', 'CONTRIB', 'PARTIC CONTR'
]
mask_account = (df['Account Number'] == account_num)
mask_pattern = df['Action'].str.upper().apply(lambda x: any(p in str(x) for p in funding_patterns))

df = df[mask_account & mask_pattern]
df['Amount ($)'].sum()

np.float64(24810.129999999997)

In [20]:
funding_patterns = ['ELECTRONIC FUNDS TRANSFER RECEIVED (CASH)']
df['Action'].str.upper().isin(funding_patterns)

553     True
562    False
545    False
531    False
530    False
       ...  
255    False
252    False
247    False
246    False
250    False
Name: Action, Length: 445, dtype: bool

### filter_positions

In [ ]:
from support_functions.data_loader import load_data
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [27]:
symbol = 'AAPL'
df = positions_df.copy()
df = df[df['Symbol'] == symbol]
df.head(4)

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
10,Z23390746,Individual,AAPL,APPLE INC,70.0,271.86,-$1.22,19030.2,-85.4,-0.45%,8270.21,+76.86%,1.52%,10759.99,$153.71,Cash


In [28]:
account_num = 'Z23390746'
df = positions_df.copy()
df = df[df['Account Number'] == account_num]
df.head(4)

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,Z23390746,Individual,912797SG3,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,200000.000,99.823,+$0.018,199646.00,36.00,+0.01%,523.33,+0.26%,15.96%,199122.67,--,Cash
1,Z23390746,Individual,FXAIX,FIDELITY 500 INDEX FUND,749.338,237.720,-$1.74,178132.62,-1303.85,-0.73%,35568.68,+24.94%,14.24%,142563.94,$190.25,Cash
2,Z23390746,Individual,FZFXX**,HELD IN MONEY MARKET,0.000,0.000,NaN,128390.20,0.00,NaN,0.00,NaN,10.27%,0.00,NaN,Cash
3,Z23390746,Individual,FSKAX,FIDELITY TOTAL MARKET INDEX FUND,658.109,186.840,-$1.42,122961.08,-934.52,-0.76%,20661.19,+20.19%,9.83%,102299.89,$155.45,Cash


### build_stock_cash_flows

#### Z23390746 AAPL

In [ ]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [4]:
symbol = 'AAPL'
account_num = 'Z23390746'

In [ ]:
filtered_hist = filter_transactions(transactions_df, account_num, symbol)
filtered_posi = filter_positions(positions_df, account_num, symbol)

In [6]:
cash_flows = []
total_invested = 0.0
current_val = filtered_posi['Current Value'].iloc[0]
for _, row in filtered_hist.iterrows():
    date = row['Run Date']
    amount = row['Amount ($)']
    flow = amount
    cash_flows.append((date, flow))
    
    # Track Invested Capital (Sum of negative flows)
    if flow < 0:
        total_invested += abs(flow)

cash_flows.append((latest_date, current_val))


In [7]:
cash_flows

[(Timestamp('2022-08-05 00:00:00'), -1481.45),
 (Timestamp('2022-08-05 00:00:00'), -163.54),
 (Timestamp('2022-08-26 00:00:00'), -1645.0),
 (Timestamp('2022-08-29 00:00:00'), -3200.0),
 (Timestamp('2022-11-01 00:00:00'), -1500.0),
 (Timestamp('2022-11-08 00:00:00'), -2770.0),
 (Timestamp('2022-11-10 00:00:00'), 11.5),
 (Timestamp('2023-02-16 00:00:00'), 16.1),
 (Timestamp('2023-05-18 00:00:00'), 16.8),
 (Timestamp('2023-08-17 00:00:00'), 16.8),
 (Timestamp('2023-11-16 00:00:00'), 16.8),
 (Timestamp('2024-02-15 00:00:00'), 16.8),
 (Timestamp('2024-05-16 00:00:00'), 17.5),
 (Timestamp('2024-08-15 00:00:00'), 17.5),
 (Timestamp('2024-11-14 00:00:00'), 17.5),
 (Timestamp('2025-02-13 00:00:00'), 17.5),
 (Timestamp('2025-05-15 00:00:00'), 18.2),
 (Timestamp('2025-08-14 00:00:00'), 18.2),
 (Timestamp('2025-11-13 00:00:00'), 18.2),
 (Timestamp('2025-12-31 00:00:00'), np.float64(19030.2))]

#### 86964 FBCGX

In [77]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [78]:
symbol = 'FBCGX'
account_num = '86964'

In [79]:
filtered_hist = filter_transactions(transactions_df, account_num, symbol)
filtered_posi = filter_positions(positions_df, account_num, symbol)

In [81]:
filtered_hist

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Price ($),Quantity,Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Asset Type


### build_account_cash_flows

#### Individual Z23390746

In [19]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [20]:
symbol = None
account_num = 'Z23390746'

In [23]:
filtered_hist = filter_transactions(transactions_df, account_num, symbol)
filtered_posi = filter_positions(positions_df, account_num, symbol)
filtered_hist['Amount ($)'].sum()

np.float64(1100000.0)

In [7]:
cash_flows = []
total_invested = 0.0
current_val = filtered_posi['Current Value'].sum()
for _, row in filtered_hist.iterrows():
    date = row['Run Date']
    amount = row['Amount ($)']
    flow = -amount
    cash_flows.append((date, flow))
    
    # Track Invested Capital (Sum of negative flows)
    total_invested -= (flow)

cash_flows.append((latest_date, current_val))

In [8]:
total_invested

1100000.0

#### ERNST & YOUNG 401(K) 86964

In [26]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = None
account_num = '86964'

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [27]:
filtered_hist = filter_transactions(transactions_df, account_num, symbol)
filtered_posi = filter_positions(positions_df, account_num, symbol)
filtered_hist['Amount ($)'].sum()

np.float64(24810.129999999997)

### xirr

In [15]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [18]:
symbol = None
account_num = 'Z23390746'

In [19]:
cash_flows, total_invested = build_account_cash_flows(transactions_df, positions_df, latest_date, account_num)

In [25]:
dates, amounts = width = zip(*cash_flows)
min_date = min(dates)
days = [(d - min_date).days for d in dates]

# Optimization function
def npv(r):
    arr = np.array(days)
    vals = np.array(amounts)
    return np.sum(vals / (1 + r)**(arr / 365.0))
    
# Check signs
pos = any(a > 0 for a in amounts)
neg = any(a < 0 for a in amounts)
res = optimize.newton(npv, 0.1, maxiter=50)
res

np.float64(0.09884212125959153)

### analyze_entity_performance

#### AAPL

In [13]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = 'AAPL'
account_num = 'Z23390746'

cash_flows, total_invested = build_stock_cash_flows(transactions_df, positions_df, latest_date, account_num, symbol)
irr = xirr(cash_flows)
irr

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


np.float64(0.19542017278306092)

#### Individual Z23390746

In [14]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = None
account_num = 'Z23390746'

cash_flows, total_invested = build_account_cash_flows(transactions_df, positions_df, latest_date, account_num)
irr = xirr(cash_flows)
irr

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


np.float64(0.09884212125959153)

#### ERNST & YOUNG 401(K) 86964

In [74]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = 'FBCGX' # FBCGX 84679P173
account_num = '86964'


Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [75]:
cash_flows, total_invested = build_stock_cash_flows(transactions_df, positions_df, latest_date, account_num, symbol)
irr = xirr(cash_flows)
irr

In [76]:
cash_flows

[(Timestamp('2025-12-31 00:00:00'), np.float64(9086.23))]

#### Cash Management (Individual) Z06872898

In [120]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = None
account_num = 'Z06872898'


Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [121]:
cash_flows, total_invested = build_stock_cash_flows(transactions_df, positions_df, latest_date, account_num, symbol)

In [123]:
total_invested

0.0

In [125]:
latest_date

Timestamp('2025-12-31 00:00:00')

## analyze_account_performance

In [3]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr, analyze_entity_performance
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [4]:
accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()
accounts_unique

,Account Number,Account Name,Current Value
0,241802439,Health Savings Account,9707.44
1,86964,ERNST & YOUNG 401(K),29217.10
2,Z06872898,Cash Management (Individual),108.17
3,Z23390746,Individual,1250701.91


In [8]:
row = accounts_unique.iloc[0]
acc_num = row['Account Number']
acc_name = row['Account Name']
curr_val = row['Current Value']
acc_num

'241802439'

In [9]:
total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=None)
irr

np.float64(0.14987364529865643)

In [10]:
total_return = curr_val-total_invested
total_return_ratio = total_return/total_invested
{
    'Account Name': acc_name,
    'Account Number': acc_num,
    'Symbol': None,
    'Asset Type': 'Account',
    'Current Value': curr_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': total_return_ratio,
    'IRR': irr
}

{'Account Name': 'Health Savings Account',
 'Account Number': '241802439',
 'Symbol': None,
 'Asset Type': 'Account',
 'Current Value': np.float64(9707.439999999999),
 'Total Invested': 8450.640000000003,
 'Total Return ($)': np.float64(1256.7999999999956),
 'Total Return (%)': np.float64(0.1487224636240563),
 'IRR': np.float64(0.14987364529865643)}

In [119]:
total_invested

8.170000000000005

In [58]:
results = []
    
accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

for _, row in accounts_unique.iterrows():
    acc_num = row['Account Number']
    acc_name = row['Account Name']
    curr_val = row['Current Value']
    if acc_name == 'ERNST & YOUNG 401(K)':
        continue
    
    total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=None)
    total_return = curr_val-total_invested
    total_return_ratio = total_return/total_invested
    
    results.append({
        'Account Name': acc_name,
        'Account Number': acc_num,
        'Symbol': None,
        'Asset Type': 'Account',
        'Current Value': curr_val,
        'Total Invested': total_invested,
        'Total Return ($)': total_return,
        'Total Return (%)': total_return_ratio,
        'IRR': irr
    })
pd.DataFrame(results)

,Account Name,Account Number,Symbol,Asset Type,Current Value,Total Invested,Total Return ($),Total Return (%),IRR
0,Health Savings Account,241802439,None,Account,9707.44,8450.64,1256.80,0.148722,0.149874
1,Cash Management (Individual),Z06872898,None,Account,108.17,8.17,100.00,12.239902,2.463594
2,Individual,Z23390746,None,Account,1250701.91,1100000.00,150701.91,0.137002,0.098842


## analyze_stock_performance

In [48]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr, analyze_entity_performance
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [50]:
results = []

accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()
accounts_unique

,Account Number,Account Name,Current Value
0,241802439,Health Savings Account,9707.44
1,86964,ERNST & YOUNG 401(K),29217.10
2,Z06872898,Cash Management (Individual),108.17
3,Z23390746,Individual,1250701.91


In [59]:
row = accounts_unique.iloc[1]
acc_num = row['Account Number']
acc_name = row['Account Name']

In [60]:
sub_positions_df = positions_df[
    (positions_df['Account Name'] == acc_name) &
    (positions_df['Asset Type'] == 'Stock')
]
sub_positions_df

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type,Asset Type
30,86964,ERNST & YOUNG 401(K),84679P173,SP 500 INDEX PL CL F,61.924,325.09,-$2.38,20130.87,-147.38,-0.73%,361.07,+1.83%,68.90%,19769.80,$319.26,NaN,Stock
31,86964,ERNST & YOUNG 401(K),FBCGX,FID BLUE CHIP GR K6,205.385,44.24,-$0.32,9086.23,-65.73,-0.72%,1558.84,+20.71%,31.10%,7527.39,$36.65,NaN,Stock


In [63]:
sub_row = sub_positions_df.iloc[1]
sub_row

Account Number                              86964
Account Name                 ERNST & YOUNG 401(K)
Symbol                                      FBCGX
Description                   FID BLUE CHIP GR K6
Quantity                                  205.385
Last Price                                  44.24
Last Price Change                          -$0.32
Current Value                             9086.23
Today's Gain/Loss Dollar                   -65.73
Today's Gain/Loss Percent                  -0.72%
Total Gain/Loss Dollar                    1558.84
Total Gain/Loss Percent                   +20.71%
Percent Of Account                         31.10%
Cost Basis Total                          7527.39
Average Cost Basis                         $36.65
Type                                          NaN
Asset Type                                  Stock
Name: 31, dtype: object

In [67]:
sub_positions_df = positions_df[
    (positions_df['Account Name'] == acc_name) &
    (positions_df['Asset Type'] == 'Stock')
]
sub_row = sub_positions_df.iloc[1]
symbol = sub_row['Symbol']
curr_val = sub_row['Current Value']


total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=symbol)
{
    'Account Name': acc_name,
    'Account Number': acc_num,
    'Symbol': symbol,
    'Asset Type': 'Stock',
    'Current Value': curr_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': total_return_ratio,
    'IRR': irr
}

{'Account Name': 'ERNST & YOUNG 401(K)',
 'Account Number': '86964',
 'Symbol': 'FBCGX',
 'Asset Type': 'Stock',
 'Current Value': np.float64(9086.23),
 'Total Invested': 0.0,
 'Total Return ($)': np.float64(4406.970000000005),
 'Total Return (%)': np.float64(0.1776278479798375),
 'IRR': None}

In [56]:
results = []

accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

for _, row in accounts_unique.iterrows():

    acc_num = row['Account Number']
    acc_name = row['Account Name']


    target_type = 'Stock'
    sub_positions_df = positions_df[
        (positions_df['Account Name'] == acc_name) &
        (positions_df['Asset Type'] == target_type)
    ]

    for _, sub_row in sub_positions_df.iterrows():
        symbol = sub_row['Symbol']
        curr_val = sub_row['Current Value']
        if symbol in ['Pending activity']:
            continue

        total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=symbol)
        results.append({
            'Account Name': acc_name,
            'Account Number': acc_num,
            'Symbol': symbol,
            'Asset Type': 'Stock',
            'Current Value': curr_val,
            'Total Invested': total_invested,
            'Total Return ($)': total_return,
            'Total Return (%)': total_return_ratio,
            'IRR': irr
        })
pd.DataFrame(results)

,Account Name,Account Number,Symbol,Asset Type,Current Value,Total Invested,Total Return ($),Total Return (%),IRR
0,Health Savings Account,241802439,FXAIX,Stock,4232.36,3560.87,4406.97,0.177628,0.167011
1,Health Savings Account,241802439,FZILX,Stock,4144.74,3836.14,4406.97,0.177628,0.300799
2,ERNST & YOUNG 401(K),86964,84679P173,Stock,20130.87,0.00,4406.97,0.177628,NaN
3,ERNST & YOUNG 401(K),86964,FBCGX,Stock,9086.23,0.00,4406.97,0.177628,NaN
4,Cash Management (Individual),Z06872898,CORE**,Stock,108.17,0.00,4406.97,0.177628,NaN
5,Individual,Z23390746,FXAIX,Stock,178132.62,142563.94,4406.97,0.177628,0.222154
6,Individual,Z23390746,FSKAX,Stock,122961.08,102299.89,4406.97,0.177628,0.167138
7,Individual,Z23390746,FSPSX,Stock,76959.48,63994.40,4406.97,0.177628,0.177839
8,Individual,Z23390746,AAPL,Stock,19030.20,10759.99,4406.97,0.177628,0.195420
9,Individual,Z23390746,MSFT,Stock,14508.60,13342.95,4406.97,0.177628,0.066716


## analyze_total_performance

In [37]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr, analyze_entity_performance
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)



accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()
accounts_unique

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


,Account Number,Account Name,Current Value
0,241802439,Health Savings Account,9707.44
1,86964,ERNST & YOUNG 401(K),29217.10
2,Z06872898,Cash Management (Individual),108.17
3,Z23390746,Individual,1250701.91


In [38]:
total_cash_flows = []
total_invested = 0
total_val = 0
for _, row in accounts_unique.iterrows():
    acc_num = row['Account Number']
    curr_val = row['Current Value']
    cash_flows, curr_invested = build_account_cash_flows(transactions_df, positions_df, latest_date, acc_num)
    total_val += curr_val
    total_invested += curr_invested
    total_cash_flows.extend(cash_flows)
irr = xirr(total_cash_flows)
irr

np.float64(0.10063390508140015)

In [42]:
total_return = total_val-total_invested
total_return_ratio = total_return/total_invested
result = [{
    'Account Name': None,
    'Account Number': None,
    'Symbol': None,
    'Asset Type': 'All',
    'Current Value': total_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': f"{total_return_ratio:.2%}",
    'IRR': f"{irr:.2%}" if irr is not None else "N/A"
}]
pd.DataFrame(result)

,Account Name,Account Number,Symbol,Asset Type,Current Value,Total Invested,Total Return ($),Total Return (%),IRR
0,None,None,None,All,1289734.62,1133360.77,156373.85,13.80%,10.06%
